# Brownian Motion Process and Black-Scholes Formula for Pricing

## Brownian Motion Process
Brownian motion, also known as Wiener process, is a continuous-time stochastic process that is widely used in finance for modeling stock prices and other financial variables. It is characterized by its properties of having independent, normally distributed increments and continuous paths.

## Black-Scholes Formula for Pricing
The Black-Scholes formula is used to calculate the theoretical price of European call and put options. The formula assumes that the stock price follows a geometric Brownian motion with constant volatility and interest rate.

### Black-Scholes Call Option Pricing Formula
$$ C = S_0 N(d_1) - K e^{-rT} N(d_2) $$
where:
- $C$ is the call option price
- $S_0$ is the current stock price
- $K$ is the strike price
- $T$ is the time to maturity
- $r$ is the risk-free interest rate
- $\sigma$ is the volatility of the stock
- $N(\cdot)$ is the cumulative distribution function of the standard normal distribution
- $d_1 = \frac{\ln(S_0 / K) + (r + 0.5 \sigma^2) T}{\sigma \sqrt{T}}$
- $d_2 = d_1 - \sigma \sqrt{T}$

### Black-Scholes Put Option Pricing Formula
$$ P = K e^{-rT} N(-d_2) - S_0 N(-d_1) $$
where:
- $P$ is the put option price
- $S_0$, $K$, $T$, $r$, $\sigma$, $N(\cdot)$, $d_1$, and $d_2$ are defined as above

## Steps to Import and Use the Datasets
1. **Import necessary libraries**: We will use `numpy` for numerical operations, `pandas` for data manipulation, `scipy.stats` for statistical functions, and `yfinance` for fetching financial data.
2. **Define the Black-Scholes functions**: We will define functions to calculate the call and put option prices using the Black-Scholes formula.
3. **Estimate parameters**: We will fetch historical stock data using `yfinance` and estimate the stock price and volatility.
4. **Calculate option prices**: Using the estimated parameters, we will calculate the call and put option prices.

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import yfinance as yf

def black_scholes_call(S, K, T, r, sigma, dividend = 0):
    c = dividend
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * np.exp(-c * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def black_scholes_put(S, K, T, r, sigma, dividend = 0):
    c = dividend
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-c * T) * norm.cdf(-d1)
    return put_price

def estimate_parameters(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data['Log Returns'] = np.log(data['Close'][ticker] / data['Close'][ticker].shift(1))
    sigma = data['Log Returns'].std() * np.sqrt(252)  # Annualized volatility
    S = data['Close'][ticker].iloc[-1]  # Last adjusted close price
    return S, sigma


In [11]:

# Example usage:
ticker = 'AAPL'
start_date = '2022-01-01'
end_date = '2023-01-01'
S, sigma = estimate_parameters(ticker, start_date, end_date)
K = 150  # Strike price
T = 1  # Time to maturity in years
r = 0.05  # Risk-free interest rate

call_price = black_scholes_call(S, K, T, r, sigma)
put_price = black_scholes_put(S, K, T, r, sigma)

print(f"Call Option Price: {call_price}")
print(f"Put Option Price: {put_price}")

[*********************100%***********************]  1 of 1 completed

Call Option Price: 12.853542860758864
Put Option Price: 27.101295769264425


In [13]:
# Checking for call-put parity
print(f"Call - Put - S + K * exp(-r * T): {call_price - put_price - S + K * np.exp(-r * T)}")

print("Call + K * exp(-r * T)", call_price + K * np.exp(-r * T))
print("Put + S", put_price + S)


Call - Put - S + K * exp(-r * T): -2.842170943040401e-14
Call + K * exp(-r * T) 155.53795653586596
Put + S 155.537956535866


## Binomial Approximation

Now assuming the stock prices move as per the binomial structure, as a close approximation to the brownian motion when number of periods approaches infinity.

To calibrate the binomial model so that its dynamics match that of the geometric Brownian motion, we need to choose $u$, $d$, and $p$, the real-world probability of an up-move, appropriately. One of the more common choices is to set:

$$ p = \frac{e^{\mu \Delta t} - d}{u - d} $$

$$ u = \exp(\sigma \sqrt{\Delta t}) $$

$$ d = \frac{1}{u} = \exp(-\sigma \sqrt{\Delta t}) $$

where $T$ is the expiration date and $\Delta t$ is the length of a period. Note then, for example, that:

$$ \mathbb{E}[S_{i+1} | S_i] = p u S_i + (1 - p) d S_i = S_i \exp(\mu \Delta t) $$

We will choose the gross risk-free rate per period, $R$, so that it corresponds to a continuously-compounded rate, $r$, in continuous time. We therefore have:

$$ R = e^{r \Delta t} $$

In [14]:
def binomial_stock_structure(S, T, sigma, n):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    # p = (np.exp(r * dt) - d) / (u - d)
    
    term_structure = np.zeros((n+1, n+1))
    for i in range(n+1):
        for j in range(i+1):
            term_structure[j, i] = S * (u ** (i-j)) * (d ** j)
    return term_structure

def binomial_option_pricing(S, K, T, RiskfreeRate, sigma, n, option_type='call', DividendYield = 0):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    riskNeutralProbability = (np.exp((RiskfreeRate - DividendYield) * dt) - d) / (u - d)
    q = riskNeutralProbability
    stock_structure = binomial_stock_structure(S, T, sigma, n)
    option_tree = np.zeros((n+1, n+1))
    for j in range(n+1):
        if option_type == 'call':
            option_tree[j, n] = max(0, stock_structure[j, n] - K)
        elif option_type == 'put':
            option_tree[j, n] = max(0, K - stock_structure[j, n])
    
    for i in range(n-1, -1, -1):
        for j in range(i+1):
            option_tree[j, i] = np.exp(-RiskfreeRate * dt) * (q * option_tree[j, i+1] + (1 - q) * option_tree[j+1, i+1])

    return option_tree[0, 0]

In [15]:
# pricing the same call option on underlying security using binomial approximation

K = 150  # Strike price
T = 1  # Time to maturity in years
r = 0.05  # Risk-free interest rate
n = 1000  # Number of time steps

ticker = 'AAPL'
start_date = '2022-01-01'
end_date = '2023-01-01'
S, sigma = estimate_parameters(ticker, start_date, end_date)

print(f"Black-Scholes Call Option Price: {black_scholes_call(S, K, T, r, sigma)}")
print(f"Binomial Call Option Price: {binomial_option_pricing(S, K, T, r, sigma, n, option_type='call')}")

# pricing the same put option on underlying security using binomial approximation
print(f"Black-Scholes Put Option Price: {black_scholes_put(S, K, T, r, sigma)}")
print(f"Binomial Put Option Price: {binomial_option_pricing(S, K, T, r, sigma, n, option_type='put')}")

[*********************100%***********************]  1 of 1 completed


Black-Scholes Call Option Price: 12.853542860758864
Binomial Call Option Price: 12.851940777719108
Black-Scholes Put Option Price: 27.101295769264425
Binomial Put Option Price: 27.099693686220796


In [16]:
## Another example for pricing given the following parameteres:

T = 0.25
n = 15
S = 100
K = 110
r = 0.02
sigma = 0.3
Div = 0.01

print(f"Black-Scholes Call Option Price: {black_scholes_call(S, K, T, r, sigma)}")
print(f"Binomial Call Option Price: {binomial_option_pricing(S, K, T, r, sigma, n, option_type='call', DividendYield=Div)}")

print(f"Black-Scholes Put Option Price: {black_scholes_put(S, K, T, r, sigma)}")
print(f"Binomial Put Option Price: {binomial_option_pricing(S, K, T, r, sigma, n, option_type='put', DividendYield=Div)}")


Black-Scholes Call Option Price: 2.6340714993612835
Binomial Call Option Price: 2.60407713296656
Black-Scholes Put Option Price: 12.085444210556346
Binomial Put Option Price: 12.305137604415611
